In [15]:
!pip install -q gradio

In [16]:
!pip install streamlit pandas scikit-learn seaborn matplotlib gradioimport gradio as gr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')


In [17]:
# Set style for matplotlib
plt.style.use('default')
sns.set_palette("husl")

# Global variables to store data and model
dataset = None
trained_model = None
X_train, X_test, y_train, y_test = None, None, None, None
scaler = StandardScaler()
label_encoders = {}

In [18]:
def load_dataset(file):
    """Load and preview dataset"""
    global dataset
    try:
        if file.name.endswith('.csv'):
            dataset = pd.read_csv(file.name)
        elif file.name.endswith(('.xlsx', '.xls')):
            dataset = pd.read_excel(file.name)
        else:
            return "❌ Please upload a CSV or Excel file", None, [], []

        # Get column names for dropdowns
        columns = dataset.columns.tolist()

        # Create a beautiful preview
        preview_html = f"""
        <div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); padding: 20px; border-radius: 10px; color: white; margin: 10px 0;">
            <h3>📊 Dataset Loaded Successfully!</h3>
            <p><strong>Shape:</strong> {dataset.shape[0]} rows × {dataset.shape[1]} columns</p>
            <p><strong>Size:</strong> {dataset.memory_usage(deep=True).sum() / 1024:.2f} KB</p>
        </div>
        """

        return preview_html, dataset.head(10), columns, columns

    except Exception as e:
        return f"❌ Error loading file: {str(e)}", None, [], []

In [19]:
def get_dataset_info():
    """Get detailed dataset information"""
    if dataset is None:
        return "⚠️ Please upload a dataset first"

    info_html = f"""
    <div style="background: linear-gradient(135deg, #11998e 0%, #38ef7d 100%); padding: 20px; border-radius: 10px; color: white;">
        <h3>📈 Dataset Information</h3>
        <div style="display: grid; grid-template-columns: repeat(auto-fit, minmax(200px, 1fr)); gap: 15px; margin-top: 15px;">
            <div style="background: rgba(255,255,255,0.2); padding: 15px; border-radius: 8px;">
                <h4>📊 Basic Stats</h4>
                <p><strong>Rows:</strong> {dataset.shape[0]}</p>
                <p><strong>Columns:</strong> {dataset.shape[1]}</p>
                <p><strong>Memory:</strong> {dataset.memory_usage(deep=True).sum() / 1024:.2f} KB</p>
            </div>
            <div style="background: rgba(255,255,255,0.2); padding: 15px; border-radius: 8px;">
                <h4>🔍 Data Types</h4>
                <p><strong>Numeric:</strong> {dataset.select_dtypes(include=[np.number]).shape[1]}</p>
                <p><strong>Categorical:</strong> {dataset.select_dtypes(include=['object']).shape[1]}</p>
                <p><strong>Missing Values:</strong> {dataset.isnull().sum().sum()}</p>
            </div>
        </div>
    </div>
    """
    return info_html

In [20]:
def train_model(features, target, model_name, test_size):
    """Train the selected model"""
    global trained_model, X_train, X_test, y_train, y_test, scaler, label_encoders

    if dataset is None:
        return "⚠️ Please upload a dataset first", None

    if not features or not target:
        return "⚠️ Please select features and target", None

    try:
        # Prepare data
        X = dataset[features].copy()
        y = dataset[target].copy()

        # Handle categorical variables
        for col in X.columns:
            if X[col].dtype == 'object':
                le = LabelEncoder()
                X[col] = le.fit_transform(X[col].astype(str))
                label_encoders[col] = le

        # Handle target variable
        if y.dtype == 'object':
            le_target = LabelEncoder()
            y = le_target.fit_transform(y.astype(str))
            label_encoders['target'] = le_target
            task_type = 'classification'
        else:
            task_type = 'regression' if len(y.unique()) > 10 else 'classification'

        # Split data
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=test_size/100, random_state=42, stratify=y if task_type == 'classification' else None
        )

        # Scale features
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)

        # Model selection
        models = {
            'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42) if task_type == 'classification' else RandomForestRegressor(n_estimators=100, random_state=42),
            'Logistic Regression': LogisticRegression(random_state=42) if task_type == 'classification' else LinearRegression(),
            'SVM': SVC(random_state=42) if task_type == 'classification' else None,
            'Decision Tree': DecisionTreeClassifier(random_state=42) if task_type == 'classification' else None,
            'K-Nearest Neighbors': KNeighborsClassifier() if task_type == 'classification' else None,
            'Naive Bayes': GaussianNB() if task_type == 'classification' else None,
            'Gradient Boosting': GradientBoostingClassifier(random_state=42) if task_type == 'classification' else None,
        }

        if model_name not in models or models[model_name] is None:
            return f"⚠️ {model_name} not available for {task_type}", None

        trained_model = models[model_name]

        # Train model
        if model_name in ['SVM', 'Logistic Regression', 'K-Nearest Neighbors', 'Naive Bayes']:
            trained_model.fit(X_train_scaled, y_train)
            y_pred = trained_model.predict(X_test_scaled)
        else:
            trained_model.fit(X_train, y_train)
            y_pred = trained_model.predict(X_test)

        # Calculate metrics
        if task_type == 'classification':
            accuracy = accuracy_score(y_test, y_pred)
            result_html = f"""
            <div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); padding: 20px; border-radius: 10px; color: white;">
                <h3>🎯 Model Training Complete!</h3>
                <div style="background: rgba(255,255,255,0.2); padding: 15px; border-radius: 8px; margin-top: 15px;">
                    <h4>📊 Results</h4>
                    <p><strong>Model:</strong> {model_name}</p>
                    <p><strong>Accuracy:</strong> {accuracy:.4f} ({accuracy*100:.2f}%)</p>
                    <p><strong>Task Type:</strong> {task_type.title()}</p>
                    <p><strong>Features Used:</strong> {len(features)}</p>
                    <p><strong>Training Samples:</strong> {len(X_train)}</p>
                    <p><strong>Test Samples:</strong> {len(X_test)}</p>
                </div>
            </div>
            """
        else:
            mse = mean_squared_error(y_test, y_pred)
            r2 = r2_score(y_test, y_pred)
            result_html = f"""
            <div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); padding: 20px; border-radius: 10px; color: white;">
                <h3>🎯 Model Training Complete!</h3>
                <div style="background: rgba(255,255,255,0.2); padding: 15px; border-radius: 8px; margin-top: 15px;">
                    <h4>📊 Results</h4>
                    <p><strong>Model:</strong> {model_name}</p>
                    <p><strong>R² Score:</strong> {r2:.4f}</p>
                    <p><strong>MSE:</strong> {mse:.4f}</p>
                    <p><strong>RMSE:</strong> {np.sqrt(mse):.4f}</p>
                    <p><strong>Task Type:</strong> {task_type.title()}</p>
                    <p><strong>Features Used:</strong> {len(features)}</p>
                </div>
            </div>
            """

        return result_html, f"✅ Model trained successfully with {accuracy*100:.2f}% accuracy!" if task_type == 'classification' else f"✅ Model trained successfully with R² = {r2:.4f}!"

    except Exception as e:
        return f"❌ Error training model: {str(e)}", None

In [21]:
def create_confusion_matrix():
    """Create confusion matrix visualization"""
    if trained_model is None or X_test is None:
        return "⚠️ Please train a model first"

    try:
        # Get predictions
        if hasattr(trained_model, 'predict'):
            if trained_model.__class__.__name__ in ['SVC', 'LogisticRegression', 'KNeighborsClassifier', 'GaussianNB']:
                y_pred = trained_model.predict(scaler.transform(X_test))
            else:
                y_pred = trained_model.predict(X_test)
        else:
            return "⚠️ Model doesn't support prediction"

        # Create confusion matrix
        cm = confusion_matrix(y_test, y_pred)

        # Create beautiful heatmap
        plt.figure(figsize=(10, 8))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                   square=True, linewidths=0.5, cbar_kws={"shrink": .8})
        plt.title('🎯 Confusion Matrix', fontsize=16, fontweight='bold', pad=20)
        plt.xlabel('Predicted Labels', fontsize=12, fontweight='bold')
        plt.ylabel('True Labels', fontsize=12, fontweight='bold')

        # Add gradient background
        plt.gca().set_facecolor('#f8f9fa')
        plt.tight_layout()

        return plt

    except Exception as e:
        print(f"Error creating confusion matrix: {str(e)}")
        return None

def create_feature_importance():
    """Create feature importance visualization"""
    if trained_model is None:
        return "⚠️ Please train a model first"

    try:
        if hasattr(trained_model, 'feature_importances_'):
            importances = trained_model.feature_importances_
            feature_names = X_train.columns

            # Create beautiful bar plot
            plt.figure(figsize=(12, 8))
            indices = np.argsort(importances)[::-1]

            colors = plt.cm.viridis(np.linspace(0, 1, len(importances)))
            bars = plt.bar(range(len(importances)), importances[indices], color=colors)

            plt.title('🔍 Feature Importance', fontsize=16, fontweight='bold', pad=20)
            plt.xlabel('Features', fontweight='bold')
            plt.ylabel('Importance', fontweight='bold')
            plt.xticks(range(len(importances)), [feature_names[i] for i in indices], rotation=45, ha='right', fontsize=10)

            # Add value labels on bars
            for i, bar in enumerate(bars):
                plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.001,
                        f'{importances[indices[i]]:.3f}', ha='center', va='bottom', fontweight='bold')

            plt.tight_layout()
            return plt
        else:
            return "⚠️ This model doesn't provide feature importance"

    except Exception as e:
        print(f"Error creating feature importance plot: {str(e)}")
        return None

def create_model_performance():
    """Create model performance visualization"""
    if trained_model is None or X_test is None:
        return "⚠️ Please train a model first"

    try:
        # Get predictions
        if trained_model.__class__.__name__ in ['SVC', 'LogisticRegression', 'KNeighborsClassifier', 'GaussianNB']:
            y_pred = trained_model.predict(scaler.transform(X_test))
        else:
            y_pred = trained_model.predict(X_test)

        # Create subplots
        fig, axes = plt.subplots(2, 2, figsize=(15, 12))
        fig.suptitle('📊 Model Performance Dashboard', fontsize=16, fontweight='bold')

        # 1. Prediction vs Actual
        axes[0, 0].scatter(y_test, y_pred, alpha=0.6, color='royalblue', edgecolors='white', linewidth=0.5)
        axes[0, 0].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
        axes[0, 0].set_xlabel('Actual Values', fontweight='bold')
        axes[0, 0].set_ylabel('Predicted Values', fontweight='bold')
        axes[0, 0].set_title('🎯 Predictions vs Actual', fontweight='bold')
        axes[0, 0].grid(True, alpha=0.3)

        # 2. Residuals
        residuals = y_test - y_pred
        axes[0, 1].scatter(y_pred, residuals, alpha=0.6, color='green', edgecolors='white', linewidth=0.5)
        axes[0, 1].axhline(y=0, color='red', linestyle='--', linewidth=2)
        axes[0, 1].set_xlabel('Predicted Values', fontweight='bold')
        axes[0, 1].set_ylabel('Residuals', fontweight='bold')
        axes[0, 1].set_title('📈 Residual Plot', fontweight='bold')
        axes[0, 1].grid(True, alpha=0.3)

        # 3. Distribution of residuals
        axes[1, 0].hist(residuals, bins=30, alpha=0.7, color='purple', edgecolor='white', linewidth=0.5)
        axes[1, 0].set_xlabel('Residuals', fontweight='bold')
        axes[1, 0].set_ylabel('Frequency', fontweight='bold')
        axes[1, 0].set_title('📊 Residual Distribution', fontweight='bold')
        axes[1, 0].grid(True, alpha=0.3)

        # 4. Q-Q plot
        from scipy import stats
        stats.probplot(residuals, dist="norm", plot=axes[1, 1])
        axes[1, 1].set_title('📉 Q-Q Plot', fontweight='bold')
        axes[1, 1].grid(True, alpha=0.3)

        plt.tight_layout()
        return fig

    except Exception as e:
        print(f"Error creating performance plots: {str(e)}")
        return None

def create_data_distribution():
    """Create data distribution visualization"""
    if dataset is None:
        return "⚠️ Please upload a dataset first"

    try:
        # Get numeric columns
        numeric_cols = dataset.select_dtypes(include=[np.number]).columns

        if len(numeric_cols) == 0:
            return "⚠️ No numeric columns found for distribution plot"

        # Create subplots
        n_cols = min(3, len(numeric_cols))
        n_rows = (len(numeric_cols) + n_cols - 1) // n_cols

        fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 5*n_rows))
        fig.suptitle('📊 Data Distribution Dashboard', fontsize=16, fontweight='bold')

        if n_rows == 1:
            axes = [axes] if n_cols == 1 else axes
        else:
            axes = axes.flatten()

        for i, col in enumerate(numeric_cols):
            if i < len(axes):
                # Create histogram with KDE
                axes[i].hist(dataset[col].dropna(), bins=30, alpha=0.7, color='skyblue', edgecolor='white', density=True)

                # Add KDE curve
                from scipy.stats import gaussian_kde
                data = dataset[col].dropna()
                if len(data) > 1:
                    kde = gaussian_kde(data)
                    x_range = np.linspace(data.min(), data.max(), 100)
                    axes[i].plot(x_range, kde(x_range), color='red', linewidth=2, label='KDE')

                axes[i].set_xlabel(col, fontweight='bold')
                axes[i].set_ylabel('Density', fontweight='bold')
                axes[i].set_title(f'📈 {col} Distribution', fontweight='bold')
                axes[i].grid(True, alpha=0.3)
                axes[i].legend()

        # Hide empty subplots
        for i in range(len(numeric_cols), len(axes)):
            axes[i].set_visible(False)

        plt.tight_layout()
        return fig

    except Exception as e:
        print(f"Error creating distribution plots: {str(e)}")
        return None

In [ ]:
# Create the Gradio interface
def create_interface():
    with gr.Blocks(theme=gr.themes.Soft(), title="🤖 AI/ML Model Training Studio") as demo:
        gr.HTML("""
        <div style="text-align: center; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); padding: 30px; border-radius: 15px; margin-bottom: 20px;">
            <h1 style="color: white; font-size: 2.5em; margin: 0; text-shadow: 2px 2px 4px rgba(0,0,0,0.3);">
                🤖 AI/ML Model Training Studio
            </h1>
            <p style="color: white; font-size: 1.2em; margin: 10px 0 0 0; opacity: 0.9;">
                Upload your dataset, select features, train models, and visualize results with beautiful charts!
            </p>
        </div>
        """)

        with gr.Tabs():
            # Dataset Upload Tab
            with gr.TabItem("📁 Dataset Upload", id="upload"):
                with gr.Row():
                    with gr.Column(scale=1):
                        file_input = gr.File(
                            label="📂 Upload Dataset (CSV/Excel)",
                            file_types=['.csv', '.xlsx', '.xls'],
                            height=100
                        )
                        dataset_info = gr.HTML()

                    with gr.Column(scale=2):
                        dataset_preview = gr.Dataframe(
                            label="📊 Dataset Preview",
                            interactive=False
                        )

                # Hidden components to store column names
                features_choices = gr.State([])
                target_choices = gr.State([])

            # Model Training Tab
            with gr.TabItem("🎯 Model Training", id="training"):
                with gr.Row():
                    with gr.Column(scale=1):
                        gr.HTML("<h3>🔧 Model Configuration</h3>")

                        features_dropdown = gr.Dropdown(
                            label="🎲 Select Features",
                            choices=[],
                            multiselect=True,
                            info="Choose input features for training"
                        )

                        target_dropdown = gr.Dropdown(
                            label="🎯 Select Target",
                            choices=[],
                            multiselect=False,
                            info="Choose the target variable to predict"
                        )

                        model_dropdown = gr.Dropdown(
                            label="🤖 Select Model",
                            choices=[
                                "Random Forest",
                                "Logistic Regression",
                                "SVM",
                                "Decision Tree",
                                "K-Nearest Neighbors",
                                "Naive Bayes",
                                "Gradient Boosting"
                            ],
                            value="Random Forest",
                            info="Choose the machine learning algorithm"
                        )

                        test_size_slider = gr.Slider(
                            label="📊 Test Size (%)",
                            minimum=10,
                            maximum=40,
                            value=20,
                            step=5,
                            info="Percentage of data for testing"
                        )

                        train_button = gr.Button(
                            "🚀 Train Model",
                            variant="primary",
                            size="lg"
                        )

                    with gr.Column(scale=2):
                        training_output = gr.HTML()
                        training_status = gr.Textbox(
                            label="📈 Training Status",
                            interactive=False
                        )

            # Visualizations Tab
            with gr.TabItem("📊 Visualizations", id="viz"):
                with gr.Row():
                    confusion_btn = gr.Button("🎯 Confusion Matrix", variant="secondary")
                    importance_btn = gr.Button("⭐ Feature Importance", variant="secondary")
                    performance_btn = gr.Button("📈 Model Performance", variant="secondary")
                    distribution_btn = gr.Button("📊 Data Distribution", variant="secondary")

                viz_output = gr.Plot(
                    label="📊 Visualization Output"
                )

        # Event handlers
        file_input.change(
            fn=load_dataset,
            inputs=[file_input],
            outputs=[dataset_info, dataset_preview, features_choices, target_choices]
        ).then(
            fn=get_dataset_info,
            outputs=[dataset_info]
        ).then(
            fn=lambda choices: (gr.Dropdown(choices=choices), gr.Dropdown(choices=choices)),
            inputs=[features_choices],
            outputs=[features_dropdown, target_dropdown]
        )

        train_button.click(
            fn=train_model,
            inputs=[features_dropdown, target_dropdown, model_dropdown, test_size_slider],
            outputs=[training_output, training_status]
        )

        confusion_btn.click(
            fn=create_confusion_matrix,
            outputs=[viz_output]
        )

        importance_btn.click(
            fn=create_feature_importance,
            outputs=[viz_output]
        )

        performance_btn.click(
            fn=create_model_performance,
            outputs=[viz_output]
        )

        distribution_btn.click(
            fn=create_data_distribution,
            outputs=[viz_output]
        )

    return demo

# Launch the interface
if __name__ == "__main__":
    demo = create_interface()
    demo.launch(
        share=True,
        debug=True,
        server_name="0.0.0.0",
        height=800
    )

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://fa4267ff35188401b2.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
